In [1]:
from pandas import DataFrame, read_csv, to_datetime
import boto3
import os
from decimal import Decimal


In [3]:
import os
from pandas import DataFrame, read_csv

def load_electricity() -> DataFrame:
    current_dir = os.path.dirname(os.path.abspath('src'))
    file_path = os.path.join(current_dir, "data", "electricity_costs.csv")

    return DataFrame(read_csv(file_path))


def load_btc_prices() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-PricesTable-1LXU143BUOBN")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df["Date"] = to_datetime(df["Date"])
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
    df.sort_values(by="Date", inplace=True)
    return df


def load_difficulties() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-DifficultyTable-DQ308ID3POT6")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df.sort_values(by="Date", inplace=True)
    return df

def load_miners() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-MiningAsicSpecsTable-6NUBQKVF3PU5")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    return df


In [4]:
electricity = load_electricity()
btc_prices = load_btc_prices()
difficulties = load_difficulties()
miners = load_miners()

In [5]:
merged_df = electricity.merge(btc_prices, on="Date")
merged_df = merged_df.merge(difficulties, on="Date")
merged_df = merged_df.merge(miners, how='cross')



In [10]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for EUR/USD exchange rate
ticker = "EURUSD=X"

# Get historical market data
data = yf.download(ticker, start="2018-01-01", end="2024-05-01")

# Display the data
# print(data.head())

# len(data)

merged_df["Date"] = to_datetime(merged_df["Date"])
merged_df = merged_df.merge(data, on="Date")


[*********************100%%**********************]  1 of 1 completed


In [29]:
merged_df["Price USD"] = merged_df["Price (EUR/MWhe)"] * merged_df["Close"]
merged_df["Price USD"] = merged_df["Price USD"].apply(lambda x: Decimal(x))

In [30]:
import pandas as pd
from decimal import Decimal
from datetime import date

BTC_BLOCK_REWARDS = (
    (date(2009, 1, 3), Decimal(50)),
    (date(2012, 11, 28), Decimal(25)),
    (date(2016, 7, 9), Decimal(12.5)),
    (date(2020, 5, 11), Decimal(6.25)),
    (date(2024, 4, 20), Decimal(3.125)),
)

merged_df["Block Reward"] = merged_df["Date"].apply(lambda x: next((reward for date, reward in reversed(BTC_BLOCK_REWARDS) if date <= x), None))

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_28606/2984215447.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  merged_df["Block Reward"] = merged_df["Date"].apply(lambda x: next((reward for date, reward in reversed(BTC_BLOCK_REWARDS) if date <= x), None))


In [31]:
def make_decimals(data_frame, cols):
    data_frame[cols] = data_frame[cols].apply(lambda x: [Decimal(y) for y in x])

make_decimals(merged_df, ["Price USD", "Price", "Difficulty", "HashRate", "Power", "Block Reward"])

In [32]:
merged_df["Price USD"] = merged_df["Price USD"].abs()
merged_df["Electricity_day_price"] = merged_df["Price USD"] * 24 * merged_df["Power"] / Decimal(1000000)
merged_df[f'Electricity_day_price'] = merged_df[f'Electricity_day_price'].apply(lambda x: Decimal(x))

In [33]:
sorted_df = merged_df.sort_values(by=["Country", "AsicID", "Date"])

In [34]:
from decimal import Decimal

unique_countries = sorted_df["Country"].unique()

for country in unique_countries:
    for i in [2, 3, 4, 5, 6]:
        sorted_df.loc[sorted_df["Country"] == country, f'Cost_{i}_months'] = sorted_df['Electricity_day_price'].rolling(window=i*30).sum()
        sorted_df.loc[sorted_df["Country"] == country, f'Cost_{i}_months'] = sorted_df[f'Cost_{i}_months'].apply(lambda x: Decimal(x))

In [35]:
sorted_df = sorted_df.dropna()


In [36]:
def add_btc_needed(df: DataFrame) -> DataFrame:
    for i in range(2, 7):
        df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
    return df

In [37]:
df = add_btc_needed(sorted_df)

In [38]:
df

,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty,AsicName,HashRate,...,Cost_2_months,Cost_3_months,Cost_4_months,Cost_5_months,Cost_6_months,BTC_needed_2_months,BTC_needed_3_months,BTC_needed_4_months,BTC_needed_5_months,BTC_needed_6_months
65156,Austria,AUT,2018-09-07,62.08,dcd36d0a-c034-4d2a-9f5d-81f38d63574c,6467.06982421875,59a83eda-1a64-4b94-9b00-c41b8fb1c3f6,7019199231177.2,Antminer S19j Pro 100TH/s,100,...,276.710640774443163536489009857177734375,384.39283664576913679411518387496471405029296875,479.2927966773977459524758160114288330078125,594.256153889864435768686234951019287109375,687.4067517817926500356406904757022857666015625,0.04278763772399364893332646955,0.05943848560382683879126121878,0.07411282229897655208532579085,0.09188955277155263262855859596,0.1062933864124212325433404113
65520,Austria,AUT,2018-09-10,64.58,c1aa79cc-ae79-41be-a079-ea60a38bf44c,6329.7001953125,1678b9e7-fc2e-40c5-90c2-295bc5ea7f49,7019199231177.2,Antminer S19j Pro 100TH/s,100,...,278.4062407369265201850794255733489990234375,386.8100674472980244900099933147430419921875,479.97677599516151758507476188242435455322265625,596.7032942234458232633187435567378997802734375,690.7339489819979689855244942009449005126953125,0.04398411175036410792645616636,0.06111032995429273192919266133,0.07582930647341107745974907113,0.09427038814023739507768180442,0.1091258555173790721910081768
65884,Austria,AUT,2018-09-11,57.70,82644aa4-9a5a-4e67-a239-d613189f91db,6321.2001953125,4976d65e-7ba7-49d3-9278-f0d1d2fe1180,7019199231177.2,Antminer S19j Pro 100TH/s,100,...,279.304213460201708585373125970363616943359375,389.3107579690532702443306334316730499267578125,480.80929037537526937740040011703968048095703125,598.02368433259971425286494195461273193359375,692.7292645239858757122419774532318115234375,0.04418531367940543418599304903,0.06158810762831835712972244297,0.07606297467558778840278119308,0.09460603459071989230729702836,0.1095882495602149723917539679
66248,Austria,AUT,2018-09-12,64.21,b3352d63-f561-48c0-b487-127adbc76a61,6351.7998046875,f5d87fdb-58ee-4edc-8b02-44a3c8860e9b,7019199231177.2,Antminer S19j Pro 100TH/s,100,...,280.8210862889828831612248905003070831298828125,391.53900027352523238732828758656978607177734375,482.66658820135404539541923440992832183837890625,599.8921295504293311751098372042179107666015625,697.418439177768277659197337925434112548828125,0.04421126214994096505267319931,0.06164221359504708510500363632,0.07598894849380420480675603541,0.09444443275868377536376062107,0.1097985548384392643883412834
66612,Austria,AUT,2018-09-13,71.28,ded6824f-3795-42ed-8f24-919128bfa06d,6517.31005859375,e5b23ab2-2c94-4d91-8932-4785dbc47abf,7019199231177.2,Antminer S19j Pro 100TH/s,100,...,284.48767146756648571681580506265163421630859375,395.3797827381591787343495525419712066650390625,485.20562931925678640254773199558258056640625,602.8251274963031391962431371212005615234375,700.8373147773885420974693261086940765380859375,0.04365108747472278884919965383,0.06066609984541243103630407661,0.07444875645888026849114619377,0.09249600250358130770241493637,0.1075347510670083549447469168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598101,Switzerland,CHE,2024-04-24,85.92,8033c36c-3b65-4d14-8040-dafd7bdd4d11,64276.8984375,5e0eae3e-5748-4753-b8bb-ca4d2c11723b,86388558925171,Antminer S17 (56Th),56,...,292.68520344775771491185878403484821319580078125,443.2709871725750190307735465466976165771484375,672.1073214621528677525930106639862060546875,878.9891902500675087139825336635112762451171875,1088.039342041805639382801018655300140380859375,0.004553505389379542041034854250,0.006896272190289206796806615854,0.01045643672610743910013498791,0.01367504051404654412256514744,0.01692737777476563761559922450
598478,Switzerland,CHE,2024-04-25,84.95,e42c9211-5c3d-49de-b9ed-a8dcec96426c,64481.70703125,7e4f2b36-d519-449f-9cf3-520188d2cff9,86388558925171,Antminer S17 (56Th),56,...,292.811136145659247631556354463100433349609375,443.94301850320130

In [39]:
df.to_csv("data/merged_data.csv", index=False)
